In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from random import randint
from time import sleep 

In [2]:
brand_df = pd.read_csv('github_data_active.csv').drop(columns=['Unnamed: 0'])

In [3]:
brand_df

,brand,url,label,website_active
0,#ladc,https://www.ladc-paris.com,NOT,True
1,'47,https://47brand.com,NOT,True
2,- contraer -,https://contraer.com,NOT,True
3,032c,https://032c.com,SUS,True
4,10 feet,https://10feet.nl,NOT,False
...,...,...,...,...
2010,dierbaar goed,https://www.dierbaargoedarnhem.nl,SUS,True
2011,pernille corydon,https://www.pernillecorydon.com,NOT,True
2012,moscow,https://svmoscow.com,NOT,True
2013,via vai,https://www.viavaifashion.com,NOT,True


In [4]:
# takes brand and creates the url to search Amazon products by brand
def create_brand_url_amazon(brand):
    brand_formatted = brand.replace(' ', '+')
    amazon_brand_url = 'https://www.amazon.com/s?k=' + brand_formatted
    return amazon_brand_url

In [5]:
brand_df['amazon_url'] = brand_df['brand'].apply(create_brand_url_amazon)

In [6]:
brand_df.head()

,brand,url,label,website_active,amazon_url
0,#ladc,https://www.ladc-paris.com,NOT,True,https://www.amazon.com/s?k=#ladc
1,'47,https://47brand.com,NOT,True,https://www.amazon.com/s?k='47
2,- contraer -,https://contraer.com,NOT,True,https://www.amazon.com/s?k=-+contraer+-
3,032c,https://032c.com,SUS,True,https://www.amazon.com/s?k=032c
4,10 feet,https://10feet.nl,NOT,False,https://www.amazon.com/s?k=10+feet


In [7]:
# Once on the search page for x brand, the function will find the xth product listing and its ASIN.
# The ASIN is appended to the Amazon domain name, creating and returning the product listing URL. 
def grab_product_listing_url(url, i):
    
    # Create Soup 
    HEADERS = ({'User-Agent':
                    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
                    'Accept-Language': 'en-US, en;q=0.5'})
    r = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(r.content, 'lxml')
    
    # Find ASIN if available 
    try:
        asin = soup.find('div', attrs= {'data-index': i}).get('data-asin')
        product_listing_url = 'https://www.amazon.com/dp/' + asin
    
    except AttributeError:
        product_listing_url = ''
    return product_listing_url
    

In [8]:
brand_list = []
product_url_list = []

# Iterate through each brand
for i,r in brand_df.iterrows():
    
    # Create 10 listings per brand
    product_count = 1 
    while product_count < 11:
        brand = r['brand'] 
        brand_url = r['amazon_url']
        
        
        product_url = grab_product_listing_url(brand_url, product_count)
        print(product_url)
        
        
        # Save information to lists
        brand_list.append(brand)
        product_url_list.append(product_url)
        
        
        product_count += 1
        
    sleep(randint(1,4))

KeyboardInterrupt: 

In [10]:
product_df = pd.DataFrame(list(zip(brand_list, product_url_list)),
               columns =['brand', 'product_url'])
product_df

,brand,product_url
0,#ladc,
1,#ladc,
2,#ladc,
3,#ladc,
4,#ladc,
...,...,...
20145,zhrill,
20146,zhrill,
20147,zhrill,
20148,zhrill,


In [18]:
product_df.to_csv('product_url_df')

In [ ]:
# TEST

In [17]:
# d = {'brand': ["nike", "adidas"], 'amazon_url': ["https://www.amazon.com/s?k=nike", "https://www.amazon.com/s?k=adidas" ]}
# df = pd.DataFrame(data=d)
# df

In [18]:
# brand_list = []
# product_url_list = []

# # Iterate through each brand
# for i,r in df.iterrows():
    
#     # Create 10 listings per brand
#     product_count = 1 
#     while product_count < 11:
#         brand = r['brand'] 
#         brand_url = r['amazon_url']
#         print(brand)
        
#         product_url = grab_product_listing_url(brand_url, product_count)
        
#         # Save information to lists
#         brand_list.append(brand)
#         product_url_list.append(product_url)
        
#         product_count += 1
        
#     sleep(randint(2,10))

In [19]:
# product_df = pd.DataFrame(list(zip(brand_list, product_url_list)),
#                columns =['brand', 'product_url'])
# product_df
